In [10]:
from vertexai.preview.generative_models import GenerativeModel
from vertexai.preview import tuning
from vertexai.preview.tuning import sft

import dill
from collections import Counter
from environment import MiniWoBEnvironment
from utils import remove_nonessentials
import json
import re
import time
import os

In [59]:
TRAINING_STATS = dill.load(open('training_stats.pkd','rb'))


from vertexai.preview.generative_models import SafetySetting,HarmCategory,HarmBlockThreshold
### 4 = BLOCK NONE
safety_settings = [SafetySetting(category=HarmCategory(0),threshold=HarmBlockThreshold(1)),
SafetySetting(category= HarmCategory(1), threshold= HarmBlockThreshold(1)),
SafetySetting(category= HarmCategory(2),threshold= HarmBlockThreshold(1)),
  SafetySetting(category= HarmCategory(3), threshold= HarmBlockThreshold(1))]

from vertexai.preview import generative_models
safety_settings = {
            generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
            generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
            generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
            generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH
        }

### Test1: Test model click-tab-2-hard with samples of same environment

In [60]:
env_type = 'click-tab-2-hard'
SAVE_TO_FOLDER = 'trajectories_test'
MODEL_DIR_NAME = 'model_click-tab-2-hard'
MAX_SAMPLES = 50

MODELS = {'click-tab-2-hard':7945776118500425728}
MODEL_PATH = 'projects/829542692869/locations/us-central1/tuningJobs/{}'.format(MODELS[env_type])

sft_tuning_job = sft.SupervisedTuningJob(MODEL_PATH)
tuned_model = GenerativeModel(sft_tuning_job.tuned_model_endpoint_name, safety_settings = safety_settings )

training_prompts = TRAINING_STATS[TRAINING_STATS.env_name == env_type].goal.values

### Check uniqueness in training dataset
for click_tab_2_hard only, all other datasets were checked before training

In [36]:
repeats = [x[0] for x in Counter(training_prompts).items() if x[1]>1]
for goal in repeats:
    print(goal)
    filenames = TRAINING_STATS[(TRAINING_STATS.env_name == env_type) & (TRAINING_STATS.goal == goal)].filename.values
    for filename in filenames:
        print(filename)
        trajectory = dill.load(open(filename,'rb'))
        for state in trajectory['states']:
            print(state['action'])
    print()

Switch between the tabs to find and click on the link "blandit".
trajectories_train/click-tab-2-hard/click-tab-2-hard015.pkd

Click on tab number 2 because the link was not found in tab 1 - CLICK_ELEMENT 8
Click on link blandit  - CLICK_ELEMENT 22
trajectories_train/click-tab-2-hard/click-tab-2-hard001.pkd

Click on tab number 2 because the link was not found in tab 1 - CLICK_ELEMENT 8
Click on tab number 3 because the link was not found in tab 2 - CLICK_ELEMENT 10
Click on tab number 4 because the link was not found in tab 3 - CLICK_ELEMENT 12
Click on tab number 5 because the link was not found in tab 4 - CLICK_ELEMENT 14
Click on tab number 6 because the link was not found in tab 5 - CLICK_ELEMENT 16
Click on link blandit  - CLICK_ELEMENT 48
trajectories_train/click-tab-2-hard/click-tab-2-hard036.pkd

Click on link blandit  - CLICK_ELEMENT 17

Switch between the tabs to find and click on the link "convallis".
trajectories_train/click-tab-2-hard/click-tab-2-hard097.pkd

Click on link

In [68]:
def save_trajectory(folder, actions, doms, images, times, env_type, reward, utterance=''):
    traj = {}
    traj['utterance'] = utterance
    traj['reward'] = reward
    traj['states'] = []
    for i in range(len(actions)):
        state = {}
        state['time'] = times[i]
        state['action'] = actions[i]
        state['dom_elements'] = doms[i]
        state['screenshot'] = images[i]
        traj['states'].append(state)
    filename,number = name_file(env_type, folder)
    if number > MAX_SAMPLES:
        print('Trajectory not saved. Already {} trajectories saved'.format(MAX_SAMPLES))
    dill.dump(traj, open(filename,'wb'))
    print('saved trajectory in file',filename)
    return traj
    
def name_file(env_name, folder):
    files = os.listdir(folder+'/'+MODEL_DIR_NAME)
    same_name = sorted([x for x in files if env_name in x])
    if same_name:
        number = int(same_name[-1][-7:-4])
        name = '{}/{}/{}{}.pkd'.format(folder,MODEL_DIR_NAME,env_name,str(number+1).zfill(3))
    else:
        name = '{}/{}/{}001.pkd'.format(folder,MODEL_DIR_NAME,env_name)
    return name,number

def create_prompt(goal,dom):
    dom = remove_nonessentials(dom)
    return "Goal: {}, DOM elements: {}".format(goal,dom).replace(
        ', dtype=float32)','').replace(
        ', dtype=int8)','').replace('array(','').replace("'","")

def test_env(env_type):
    actions, doms, images, times = [], [], [], []
    
    def execute_action(action_type, ref, text, action_txt):
        action = env.create_action(action_type, ref=ref, text=text)
        observation, reward, terminated, truncated, info = env.step(action)
        dom = observation['dom_elements']
        image = observation['screenshot']
        actions.append(action_txt)
        doms.append(dom)
        images.append(image)
        times.append(int(time.time()-start_time))
        print('Done:',info['done'])
        return reward, info['done']
        
    env = MiniWoBEnvironment(env_type, wait_ms=500,render_mode='human')
    start_time = time.time()
    observation, info = env.reset()
    
    goal = observation['utterance']
    print('Prompt:',goal)
    if goal in training_prompts:
        env.close()
        raise Exception('Prompt already used for training')

    actions.append('')
    doms.append(observation['dom_elements'])
    images.append(observation['screenshot'])
    times.append(int(time.time()-start_time))
    
    prompt = create_prompt(goal,doms[-1])
    return dom
    done = False
    while not done:
        response = tuned_model.generate_content(prompt).text
        print('Response:',response)
        if 'CLICK_ELEMENT' in response:
            action_type = 'CLICK_ELEMENT'
            text = ''
        else:
            action_type = 'TYPE_TEXT'
            return observation
            text = observation['fields']
        ref = int(re.search('{} ([0-9]+)'.format(action_type),response).group(1))
        reward, done = execute_action(action_type, ref, text, '{} {}'.format(action_type, ref))
        
        prompt = create_prompt(goal,doms[-1])
    print('Reward:',reward)
    env.close()
    save_trajectory(SAVE_TO_FOLDER, actions, doms, images, times, env_type, reward, goal)

### Test 2: Test all environments with model trained on partial data, missing search-engine and click-checkboxes-soft

In [178]:
TEXT_FIELDS = {'click-checkboxes-soft':[],
 'click-tab-2-hard':[],
 'social-media':[],
 'email-inbox':['message','to'],
 'social-media-some':[],
 'tic-tac-toe':[],
 'use-autocomplete':['start'],
 'book-flight':['from','to'],
 'choose-date':[],
 'search-engine':['query']}

env_type = 'click-tab-2-hard'
if env_type == 'tic-tac-toe':
    training_prompts = []
else:
    training_prompts = TRAINING_STATS[TRAINING_STATS.env_name == env_type].goal.values

In [179]:
SAVE_TO_FOLDER = 'trajectories_test'
MODEL_DIR_NAME = 'model_partial'
MAX_SAMPLES = 50

MODELS = {'click-tab-2-hard': 7945776118500425728
         , 'partial': 3725331521607827456}
MODEL_PATH = 'projects/829542692869/locations/us-central1/tuningJobs/{}'.format(MODELS['partial'])

sft_tuning_job = sft.SupervisedTuningJob(MODEL_PATH)
tuned_model = GenerativeModel(sft_tuning_job.tuned_model_endpoint_name, safety_settings = safety_settings )

In [183]:
def ref2text(ref, dom):
    for element in dom:
        if element['ref'] == ref:
            text = element['text']
            return text

def save_trajectory(folder, actions, doms, images, times, env_type, reward, utterance, model_dir_name):
    traj = {}
    traj['utterance'] = utterance
    traj['reward'] = reward
    traj['states'] = []
    for i in range(len(actions)):
        state = {}
        state['time'] = times[i]
        state['action'] = actions[i]
        state['dom_elements'] = doms[i]
        state['screenshot'] = images[i]
        traj['states'].append(state)
    filename,number = name_file(env_type, folder,model_dir_name)
    if number > MAX_SAMPLES:
        print('Trajectory not saved. Already {} trajectories saved'.format(MAX_SAMPLES))
        return False
    dill.dump(traj, open(filename,'wb'))
    print('saved trajectory in file',filename)
    return True
    
def name_file(env_name, folder,model_dir_name):
    files = os.listdir(folder+'/'+model_dir_name)
    same_name = sorted([x for x in files if env_name in x])
    if same_name:
        number = int(same_name[-1][-7:-4]) 
    else:
        number = 1
    name = '{}/{}/{}{}.pkd'.format(folder,model_dir_name,env_name,str(number+1).zfill(3))
    return name,number

def test_env1(env_type, model_dir_name):
    actions, doms, images, times = [], [], [], []
    
    def execute_action(action_type, ref, text, action_txt):
        action = env.create_action(action_type, ref=ref, text=text)
        observation, reward, terminated, truncated, info = env.step(action)
        dom = observation['dom_elements']
        image = observation['screenshot']
        actions.append(action_txt)
        doms.append(dom)
        images.append(image)
        times.append(int(time.time()-start_time))
        print('Done:',info['done'])
        return reward, info['done']
        
    env = MiniWoBEnvironment(env_type, wait_ms=500,render_mode='human')
    start_time = time.time()
    observation, info = env.reset()
    
    goal = observation['utterance']
    fields = dict(observation['fields'])
    print('Prompt:',goal)

    texts = []
    for key in TEXT_FIELDS[env_type]:
        if fields.get(key,False):
            texts.append(fields[key])
    text_index = 0
    
    if goal in training_prompts:
        env.close()
        raise Exception('Prompt already used for training')

    actions.append('')
    doms.append(observation['dom_elements'])
    images.append(observation['screenshot'])
    times.append(int(time.time()-start_time))
    
    prompt = create_prompt(goal,doms[-1])

    done = False
    while not done:
        response = tuned_model.generate_content(prompt).text
        print('Response:',response)
        if 'CLICK_ELEMENT' in response:
            action_type = 'CLICK_ELEMENT'
            text = ''
        elif 'TYPE_TEXT' in response:
            action_type = 'TYPE_TEXT'
            text = texts[text_index]
            if text not in response:
                text = ''
        ref = int(re.search('{} ([0-9]+)'.format(action_type),response).group(1))
        reward, done = execute_action(action_type, ref, text, '{} {}'.format(action_type, ref))
        if 'TYPE_TEXT' in response:
            if ref2text(ref, doms[-1]) == text:
                text_index +=1
        prompt = create_prompt(goal,doms[-1])
    print('Reward:',reward)
    env.close()
    save_trajectory(SAVE_TO_FOLDER, actions, doms, images, times, env_type, reward, goal, model_dir_name)
test_env1(env_type,MODEL_DIR_NAME)

Prompt: Switch between the tabs to find and click on the link "adipiscing".


Exception: Prompt already used for training